In [1]:
import json

# Count elements in test.jsonl
test_count = 0
with open('test.jsonl', 'r') as f:
    for line in f:
        test_count += 1


# Count elements in train.jsonl
train_count = 0
with open('train.jsonl', 'r') as f:
    for line in f:
        train_count += 1

print(f"Number of elements in test.jsonl: {test_count}")
print(f"Number of elements in train.jsonl: {train_count}")


Number of elements in test.jsonl: 2000
Number of elements in train.jsonl: 31716
